<a href="https://colab.research.google.com/github/samueleborgognoni/kmeans_nn_ILF_london_smart_meters/blob/main/6_ADS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly Detection System

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

drive_folder_path = '/content/drive/MyDrive/-Career📚/_UNIVERSITY/__Macchine_ed_azionamenti_elettrici/_Progetto_MAE/mae_proj/'

Mounted at /content/drive


In [11]:
# Import previously extracted data for CLUSTER 0
X_ilf_train = pd.read_csv(drive_folder_path + 'data/X_ilf_train.csv')
X_ilf_test = pd.read_csv(drive_folder_path + 'data/X_ilf_test.csv')
y_ilf_train = pd.read_csv(drive_folder_path + 'data/y_ilf_train.csv')
y_ilf_test = pd.read_csv(drive_folder_path + 'data/y_ilf_test.csv')

y_ilf_test_attacked = pd.read_csv(drive_folder_path + 'data/y_ilf_test_attacked.csv')

# --
y_ilf_test.drop(columns=y_ilf_test.columns[0], inplace=True)
y_ilf_train.drop(columns=y_ilf_train.columns[0], inplace=True)

In [40]:
# SELECT ONE SMART METER

meter_id = 'MAC000002' # <-- choose a meter LCLid

y_test_attacked_meter = y_ilf_test_attacked[y_ilf_test_attacked['LCLid'] == meter_id].drop(columns='LCLid')
y_test_meter = y_ilf_test[y_ilf_test['LCLid'] == meter_id].drop(columns='LCLid')
y_train_meter = y_ilf_train[y_ilf_train['LCLid'] == meter_id].drop(columns='LCLid')

X_train_meter = X_ilf_train[X_ilf_train['LCLid'] == meter_id].drop(columns=['energy_mean_cluster','LCLid', 'lag_1', 'lag_24', 'tstp'])
X_test_meter = X_ilf_test[X_ilf_test['LCLid'] == meter_id].drop(columns=['energy_mean_cluster','LCLid', 'lag_1', 'lag_24', 'tstp'])


In [41]:
# Concat X and y df, since OCSVM is (semi-)unsupervised method and needs onlt one dataset
df_train_meter = pd.concat([X_train_meter, y_train_meter], axis=1).set_index('tstp')
df_test_meter = pd.concat([X_test_meter, y_test_attacked_meter], axis=1).set_index('tstp')

In [42]:
# Show the new dataset
df_test_meter

,hour_sin,day_sin,month_sin,energy(kWh)
tstp,,,,
2013-06-28 09:00:00,7.071068e-01,-0.433884,1.224647e-16,0.5280
2013-06-28 10:00:00,5.000000e-01,-0.433884,1.224647e-16,0.2448
2013-06-28 11:00:00,2.588190e-01,-0.433884,1.224647e-16,0.2448
2013-06-28 12:00:00,1.224647e-16,-0.433884,1.224647e-16,0.2544
2013-06-28 13:00:00,-2.588190e-01,-0.433884,1.224647e-16,1.0620
...,...,...,...,...
2014-02-27 20:00:00,-8.660254e-01,0.433884,8.660254e-01,2.2590
2014-02-27 21:00:00,-7.071068e-01,0.433884,8.660254e-01,1.0000
2014-02-27 22:00:00,-5.000000e-01,0.433884,8.660254e-01,1.7660


### Feature extraction

In [43]:
from scipy.stats import kurtosis

def extract_features_adc(df_meter, window=24):
    '''Extract statistic features from X_train and X_test on over
    the selected time windows'''

    # Select the energy col
    s = df_meter['energy(kWh)']

    # Rolling statistics
    df_meter[f'mean_{window}'] = s.rolling(window).mean()
    df_meter[f'std_{window}'] = s.rolling(window).std()
    df_meter[f'rms_{window}'] = np.sqrt((s**2).rolling(window).mean())
    df_meter[f'mean_abs_{window}'] = (np.abs(s)).rolling(window).mean()
    df_meter[f'max_abs_{window}'] = (np.abs(s)).rolling(window).max()

    # Kurtosis
    df_meter[f'kurtosis_{window}'] = s.rolling(window).apply(
        lambda x: kurtosis(x), raw=True
    )

    # Derived factors
    df_meter[f'crest_factor_{window}'] = df_meter[f'max_abs_{window}'] / df_meter[f'rms_{window}']
    df_meter[f'impulse_factor_{window}'] = df_meter[f'max_abs_{window}'] / df_meter[f'mean_abs_{window}']
    df_meter[f'shape_factor_{window}'] = df_meter[f'rms_{window}'] / df_meter[f'mean_abs_{window}']


    # drop first NaN samples due to windowing
    df_meter.dropna(inplace=True)

    return df_meter




In [44]:
extracted_df_train_meter = extract_features_adc(df_train_meter, window=24)
extracted_df_test_meter = extract_features_adc(df_test_meter, window=24)


In [45]:
extracted_df_train_meter

,hour_sin,day_sin,month_sin,energy(kWh),mean_24,std_24,rms_24,mean_abs_24,max_abs_24,kurtosis_24,crest_factor_24,impulse_factor_24,shape_factor_24
tstp,,,,,,,,,,,,,
2012-10-13 23:00:00,-0.258819,-0.974928,-8.660254e-01,0.509,0.461958,0.359322,0.580636,0.461958,1.848,7.888832,3.182716,4.000361,1.256901
2012-10-14 00:00:00,0.000000,-0.781831,-8.660254e-01,0.428,0.457625,0.359068,0.577043,0.457625,1.848,8.061297,3.202536,4.038241,1.260951
2012-10-14 01:00:00,0.258819,-0.781831,-8.660254e-01,0.314,0.448583,0.359871,0.570384,0.448583,1.848,8.262353,3.239922,4.119636,1.271523
2012-10-14 02:00:00,0.500000,-0.781831,-8.660254e-01,0.208,0.442792,0.362685,0.567559,0.442792,1.848,8.113995,3.256047,4.173520,1.281775
2012-10-14 03:00:00,0.707107,-0.781831,-8.660254e-01,0.206,0.439708,0.364439,0.566238,0.439708,1.848,8.007260,3.263647,4.202786,1.287758
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-06-28 04:00:00,0.866025,-0.433884,1.224647e-16,0.205,0.358833,0.242363,0.430179,0.358833,1.079,2.273181,2.508259,3.006967,1.198827
2013-06-28 05:00:00,0.965926,-0.433884,1.224647e-16,0.206,0.358958,0.242280,0.430238,0.358958,1.079,2.275947,2.507912,3.005920,1.198575
2013-06-28 06:00:00,1.000000,-0.433884,1.224647e-16,0.211,0.359292,0.242062,0.430399,0.359292,1.079,2.283125,2.506978,3.003131,1.197909


In [79]:
from sklearn.preprocessing import StandardScaler

# Standardize data
scaler = StandardScaler()
scaled_df_train_meter = scaler.fit_transform(extracted_df_train_meter)
scaled_df_test_meter = scaler.transform(extracted_df_test_meter)


### PCA

In [91]:
from sklearn.decomposition import PCA

# Don't specify the number of PCA components, but the variance to be represented
pca = PCA(n_components=0.95)
pca_train_meter = pca.fit_transform(scaled_df_train_meter)
pca_test_meter = pca.transform(scaled_df_test_meter)

print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))

[0.40815439 0.22174743 0.09617816 0.07659164 0.07417899 0.05665955
 0.04285391]
0.9763640780954022


### OC-SVM Anomaly Detection algorithm

In [92]:
from sklearn.svm import OneClassSVM

# OCSVM TRAINING (CLEAN)
ocsvm = OneClassSVM(nu=0.1, kernel='rbf', gamma='scale') # max 10% allowed outliers
ocsvm.fit(pca_train_meter)


# TEST predict class
y_pred = ocsvm.predict(pca_test_meter)  # -1=attack, +1=clean

print(f"Attacks detected: {np.sum(y_pred==-1)} / {len(y_pred)} = {(np.sum(y_pred==-1)/len(y_pred))*100:.1f}% of the samples")


Attacks detected: 1515 / 5849 = 25.9% of the samples


In [60]:
y_target = np.where(y_ilf_test == y_ilf_test_attacked, 1, -1)
n_attacks = np.sum(y_target == -1)
print(f"Attacks detected for meter {meter_id}:\n{n_attacks}/{len(y_target)} ({round(n_attacks/len(y_target)*100, 2)}%)")

Attacks detected for meter MAC000002:
163178/643974 (25.34%)


### Classification results + confusion matrix

In [63]:
y_pred_bin = (y_pred == 1).astype(int)
y_true_bin = (y_target == 1).astype(int)

In [62]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# 1. Convert OCSVM labels: 1=normal, 0=attack
y_pred_bin = (y_pred == 1).astype(int)
y_true_bin = (y_target == 1).astype(int)

# CONFUSION MATRIX
cm = confusion_matrix(y_true_bin, y_pred_bin)
print("Confusion Matrix:")
print(cm)

# Complete metrics
tn, fp, fn, tp = cm.ravel()

accuracy = accuracy_score(y_true_bin, y_pred_bin)
precision = precision_score(y_true_bin, y_pred_bin)
recall = recall_score(y_true_bin, y_pred_bin)
specificity = tn / (tn + fp)
f1 = f1_score(y_true_bin, y_pred_bin)

print("\nCLASSIFICATION METRICS (ATTACK or NOT):")
print(f"Accuracy:   {accuracy*100:.1f}%")
print(f"Sensitivity: {recall*100:.1f}%")
print(f"Precision:  {precision*100:.1f}%")
print(f"Specificity:{specificity*100:.1f}%")
print(f"F1-Score:   {f1:.4f}")

# Plot
plt.figure(figsize=(10, 4))
# Confusion Matrix heatmap
plt.subplot(1, 2, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Attack', 'Clean'],
            yticklabels=['Attack', 'Clean'])
plt.title('Confusion Matrix\nADS Cluster 0', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
# Metrics bar plot
plt.subplot(1, 2, 2)
metrics = ['Accuracy', 'Sensitivity', 'Precision', 'Specificity']
values = [accuracy, recall, precision, specificity]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
bars = plt.bar(metrics, values, color=colors, alpha=0.8, edgecolor='black')
plt.ylim(0, 1.05)
plt.title('ADS Performance Metrics', fontsize=14, fontweight='bold')
plt.ylabel('Score')

# Numeric values on the bars
for bar, val in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{val:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('cluster0_ads_fig18.png', dpi=300, bbox_inches='tight')
plt.show()



ValueError: Found input variables with inconsistent numbers of samples: [643974, 5849]

In [ ]:
# Features DFD (per cluster):
features = ['mean', 'std', 'rms', 'crest_factor', 'impulse_factor', 'kurtosis', 'shape_factor']

# 1. Estrai features da dati puliti del cluster
X_clean = extract_dfd_features(cluster_data_clean)[features]

# 2. PCA (riduci dimensionalità)
pca = PCA(n_components=0.95)  # 95% varianza
X_pca = pca.fit_transform(StandardScaler().fit_transform(X_clean))

# 3. Train OCSVM
ocsvm = OneClassSVM(kernel='rbf', nu=0.05)  # nu = % outliers attesi
ocsvm.fit(X_pca)

# 4. Test su dati "attaccati"
X_attacked_pca = pca.transform(StandardScaler().transform(extract_dfd_features(attacked_data)))
anomalies = ocsvm.predict(X_attacked_pca)  # -1 = anomaly!
